In [ ]:
!git clone https://github.com/nerdimite/ntm


Cloning into 'ntm'...
remote: Enumerating objects: 23, done.
remote: Counting objects: 100% (23/23), done.
remote: Compressing objects: 100% (21/21), done.
remote: Total 23 (delta 5), reused 19 (delta 2), pack-reused 0
Unpacking objects: 100% (23/23), 260.53 KiB | 1.33 MiB/s, done.


In [ ]:
cd ntm

/content/ntm


In [ ]:
import json
import torch
from torch import nn, optim
from torchvision import transforms as T
import numpy as np
import pandas as pd
from google.colab import files
from ntm import NTM
import argparse
from tqdm import tqdm
from sklearn.model_selection import train_test_split

In [ ]:
task_params = json.load(open('configs/copy.json'))
task_params['task'] = 'predict'
task_params['min_seq_len'] = 1
task_params['max_seq_len'] = 14

In [ ]:
ntm = NTM(input_dim=task_params['seq_width'] + 2,
          output_dim=task_params['seq_width'],
          ctrl_dim=task_params['controller_size'],
          memory_units=task_params['memory_units'],
          memory_unit_size=task_params['memory_unit_size'],
          num_heads=task_params['num_heads'])

In [ ]:
uploaded = files.upload()


Saving MBTA-Bus-Arrival-Departure-Times_2022-01.csv to MBTA-Bus-Arrival-Departure-Times_2022-01.csv


In [ ]:
data = pd.read_csv("MBTA-Bus-Arrival-Departure-Times_2022-01.csv",parse_dates=["service_date"])
data.head()



,service_date,route_id,direction_id,half_trip_id,stop_id,time_point_id,time_point_order,point_type,standard_type,scheduled,actual,scheduled_headway,headway
0,2022-01-01,01,Inbound,54137355.0,110,hhgat,1,Startpoint,Schedule,1900-01-01 06:05:00.000,1900-01-01 06:24:56.000,NaN,NaN
1,2022-01-01,01,Inbound,54137355.0,67,maput,2,Midpoint,Schedule,1900-01-01 06:09:00.000,1900-01-01 06:27:46.000,NaN,NaN
2,2022-01-01,01,Inbound,54137355.0,72,cntsq,3,Midpoint,Schedule,1900-01-01 06:12:00.000,1900-01-01 06:32:16.000,NaN,NaN
3,2022-01-01,01,Inbound,54137355.0,75,mit,4,Midpoint,Schedule,1900-01-01 06:15:00.000,1900-01-01 06:35:03.000,NaN,NaN
4,2022-01-01,01,Inbound,54137355.0,79,hynes,5,Midpoint,Schedule,1900-01-01 06:19:00.000,1900-01-01 06:38:10.000,NaN,NaN


In [ ]:
data = data[0:1000]

In [ ]:
desired = ['service_date','route_id','half_trip_id','time_point_order','point_type','scheduled','actual']
data3 = data[desired]

data3['scheduled_time']=pd.to_datetime(data3['scheduled'], errors='coerce')
tindex=pd.DatetimeIndex(data3.scheduled_time)
data3['scheduled_time']=tindex.strftime('%H:%M:%S')
data3['converted_scheduled_time']=pd.to_timedelta(data3['scheduled_time']).dt.total_seconds()

data3['actual_time']=pd.to_datetime(data3['actual'], errors='coerce')
tindex=pd.DatetimeIndex(data3.actual_time)
data3['actual_time']=tindex.strftime('%H:%M:%S')
data3['converted_actual_time']=pd.to_timedelta(data3['actual_time']).dt.total_seconds()





print(data3)

    service_date route_id  half_trip_id  time_point_order  point_type  \
0     2022-01-01       01    54137355.0                 1  Startpoint   
1     2022-01-01       01    54137355.0                 2    Midpoint   
2     2022-01-01       01    54137355.0                 3    Midpoint   
3     2022-01-01       01    54137355.0                 4    Midpoint   
4     2022-01-01       01    54137355.0                 5    Midpoint   
..           ...      ...           ...               ...         ...   
995   2022-01-01       01    54137258.0                 3    Midpoint   
996   2022-01-01       01    54137212.0                 8    Midpoint   
997   2022-01-01       01    54137212.0                 9    Endpoint   
998   2022-01-01       01    54137244.0                 6    Midpoint   
999   2022-01-01       01    54137278.0                 1  Startpoint   

                   scheduled                   actual scheduled_time  \
0    1900-01-01 06:05:00.000  1900-01-01 06:24:56.0

<ipython-input-10-bc6a570c3a19>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data3['scheduled_time']=pd.to_datetime(data3['scheduled'], errors='coerce')
<ipython-input-10-bc6a570c3a19>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data3['scheduled_time']=tindex.strftime('%H:%M:%S')
<ipython-input-10-bc6a570c3a19>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: 

In [ ]:
day_of_week = data3['service_date'].dt.day_name()
data3.loc[:, 'day_of_week'] = day_of_week
data3.head(50)

<ipython-input-11-6f2fb0e3900f>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data3.loc[:, 'day_of_week'] = day_of_week


,service_date,route_id,half_trip_id,time_point_order,point_type,scheduled,actual,scheduled_time,converted_scheduled_time,actual_time,converted_actual_time,day_of_week
0,2022-01-01,01,54137355.0,1,Startpoint,1900-01-01 06:05:00.000,1900-01-01 06:24:56.000,06:05:00,21900.0,06:24:56,23096.0,Saturday
1,2022-01-01,01,54137355.0,2,Midpoint,1900-01-01 06:09:00.000,1900-01-01 06:27:46.000,06:09:00,22140.0,06:27:46,23266.0,Saturday
2,2022-01-01,01,54137355.0,3,Midpoint,1900-01-01 06:12:00.000,1900-01-01 06:32:16.000,06:12:00,22320.0,06:32:16,23536.0,Saturday
3,2022-01-01,01,54137355.0,4,Midpoint,1900-01-01 06:15:00.000,1900-01-01 06:35:03.000,06:15:00,22500.0,06:35:03,23703.0,Saturday
4,2022-01-01,01,54137355.0,5,Midpoint,1900-01-01 06:19:00.000,1900-01-01 06:38:10.000,06:19:00,22740.0,06:38:10,23890.0,Saturday
5,2022-01-01,01,54137355.0,6,Midpoint,1900-01-01 06:21:00.000,1900-01-01 06:42:40.000,06:21:00,22860.0,06:42:40,24160.0,Saturday
6,2022-01-01,01,54137355.0,7,Midpoint,1900-01-01 06:25:00.000,1900-01-01 06:45:39.000,06:25:00,23100.0,06:45:39,24339.0,Saturday
7,2022-01-01,01,54137203.0,1,Startpoint,1900-01-01 06:25:00.000,1900-01-01 06:25:53.000,06:25:00,23100.0,06:25:53,23153.0,Saturday
8,2022-01-01,01,54137203.0,2,Midpoint,1900-01-01 06:29:00.000,1900-01-01 06:27:52.000,06:29:00,23340.0,06:27:52,23272.0,Saturday
9,2022-01-01,01,54137355.0,8,Midpoint,1900-01-01 06:29:00.000,1900-01-01 06:49:47.000,06:29:00,23340.0,06:49:47,24587.0,Saturday


In [ ]:
new_df = data3.sort_values(by=['service_date','route_id','half_trip_id'])
new_df.head(50)

,service_date,route_id,half_trip_id,time_point_order,point_type,scheduled,actual,scheduled_time,converted_scheduled_time,actual_time,converted_actual_time,day_of_week
729,2022-01-01,01,54137202.0,1,Startpoint,1900-01-01 05:55:00.000,1900-01-01 05:59:50.000,05:55:00,21300.0,05:59:50,21590.0,Saturday
730,2022-01-01,01,54137202.0,2,Midpoint,1900-01-01 05:56:00.000,1900-01-01 06:00:49.000,05:56:00,21360.0,06:00:49,21649.0,Saturday
731,2022-01-01,01,54137202.0,3,Midpoint,1900-01-01 06:02:00.000,1900-01-01 06:07:03.000,06:02:00,21720.0,06:07:03,22023.0,Saturday
732,2022-01-01,01,54137202.0,4,Midpoint,1900-01-01 06:06:00.000,1900-01-01 06:10:59.000,06:06:00,21960.0,06:10:59,22259.0,Saturday
733,2022-01-01,01,54137202.0,5,Midpoint,1900-01-01 06:09:00.000,1900-01-01 06:14:04.000,06:09:00,22140.0,06:14:04,22444.0,Saturday
736,2022-01-01,01,54137202.0,6,Midpoint,1900-01-01 06:12:00.000,1900-01-01 06:16:37.000,06:12:00,22320.0,06:16:37,22597.0,Saturday
737,2022-01-01,01,54137202.0,7,Midpoint,1900-01-01 06:16:00.000,1900-01-01 06:19:55.000,06:16:00,22560.0,06:19:55,22795.0,Saturday
739,2022-01-01,01,54137202.0,8,Midpoint,1900-01-01 06:19:00.000,1900-01-01 06:22:31.000,06:19:00,22740.0,06:22:31,22951.0,Saturday
740,2022-01-01,01,54137202.0,9,Endpoint,1900-01-01 06:20:00.000,1900-01-01 06:23:08.000,06:20:00,22800.0,06:23:08,22988.0,Saturday
7,2022-01-01,01,54137203.0,1,Startpoint,1900-01-01 06:25:00.000,1900-01-01 06:25:53.000,06:25:00,23100.0,06:25:53,23153.0,Saturday


In [ ]:
new_df.head(50)

,service_date,route_id,half_trip_id,time_point_order,point_type,scheduled,actual,scheduled_time,converted_scheduled_time,actual_time,converted_actual_time,day_of_week
729,2022-01-01,01,54137202.0,1,Startpoint,1900-01-01 05:55:00.000,1900-01-01 05:59:50.000,05:55:00,21300.0,05:59:50,21590.0,Saturday
730,2022-01-01,01,54137202.0,2,Midpoint,1900-01-01 05:56:00.000,1900-01-01 06:00:49.000,05:56:00,21360.0,06:00:49,21649.0,Saturday
731,2022-01-01,01,54137202.0,3,Midpoint,1900-01-01 06:02:00.000,1900-01-01 06:07:03.000,06:02:00,21720.0,06:07:03,22023.0,Saturday
732,2022-01-01,01,54137202.0,4,Midpoint,1900-01-01 06:06:00.000,1900-01-01 06:10:59.000,06:06:00,21960.0,06:10:59,22259.0,Saturday
733,2022-01-01,01,54137202.0,5,Midpoint,1900-01-01 06:09:00.000,1900-01-01 06:14:04.000,06:09:00,22140.0,06:14:04,22444.0,Saturday
736,2022-01-01,01,54137202.0,6,Midpoint,1900-01-01 06:12:00.000,1900-01-01 06:16:37.000,06:12:00,22320.0,06:16:37,22597.0,Saturday
737,2022-01-01,01,54137202.0,7,Midpoint,1900-01-01 06:16:00.000,1900-01-01 06:19:55.000,06:16:00,22560.0,06:19:55,22795.0,Saturday
739,2022-01-01,01,54137202.0,8,Midpoint,1900-01-01 06:19:00.000,1900-01-01 06:22:31.000,06:19:00,22740.0,06:22:31,22951.0,Saturday
740,2022-01-01,01,54137202.0,9,Endpoint,1900-01-01 06:20:00.000,1900-01-01 06:23:08.000,06:20:00,22800.0,06:23:08,22988.0,Saturday
7,2022-01-01,01,54137203.0,1,Startpoint,1900-01-01 06:25:00.000,1900-01-01 06:25:53.000,06:25:00,23100.0,06:25:53,23153.0,Saturday


In [ ]:
desired = ['route_id','half_trip_id','time_point_order','point_type','converted_scheduled_time','converted_actual_time','day_of_week']
df = new_df[desired]
df.head(10)

,route_id,half_trip_id,time_point_order,point_type,converted_scheduled_time,converted_actual_time,day_of_week
729,01,54137202.0,1,Startpoint,21300.0,21590.0,Saturday
730,01,54137202.0,2,Midpoint,21360.0,21649.0,Saturday
731,01,54137202.0,3,Midpoint,21720.0,22023.0,Saturday
732,01,54137202.0,4,Midpoint,21960.0,22259.0,Saturday
733,01,54137202.0,5,Midpoint,22140.0,22444.0,Saturday
736,01,54137202.0,6,Midpoint,22320.0,22597.0,Saturday
737,01,54137202.0,7,Midpoint,22560.0,22795.0,Saturday
739,01,54137202.0,8,Midpoint,22740.0,22951.0,Saturday
740,01,54137202.0,9,Endpoint,22800.0,22988.0,Saturday
7,01,54137203.0,1,Startpoint,23100.0,23153.0,Saturday


In [ ]:
one_hot_encoded_data = pd.get_dummies(df, columns = ['route_id','half_trip_id','time_point_order','point_type','day_of_week'])

In [ ]:
print(one_hot_encoded_data)

     converted_scheduled_time  converted_actual_time  route_id_01  \
729                   21300.0                21590.0            1   
730                   21360.0                21649.0            1   
731                   21720.0                22023.0            1   
732                   21960.0                22259.0            1   
733                   22140.0                22444.0            1   
..                        ...                    ...          ...   
581                   76680.0                76283.0            1   
584                   76920.0                76446.0            1   
587                   77220.0                76559.0            1   
590                   77460.0                76810.0            1   
591                   77520.0                76848.0            1   

     half_trip_id_54137202.0  half_trip_id_54137203.0  \
729                        1                        0   
730                        1                        0   


In [ ]:
Y = one_hot_encoded_data.pop('converted_actual_time')
X = np.array(one_hot_encoded_data.to_numpy(),dtype = "float32")

#X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size=0.20)
one_hot_encoded_data['converted_actual_time'] = Y
print(one_hot_encoded_data.head(10))

     converted_scheduled_time  route_id_01  half_trip_id_54137202.0  \
729                   21300.0            1                        1   
730                   21360.0            1                        1   
731                   21720.0            1                        1   
732                   21960.0            1                        1   
733                   22140.0            1                        1   
736                   22320.0            1                        1   
737                   22560.0            1                        1   
739                   22740.0            1                        1   
740                   22800.0            1                        1   
7                     23100.0            1                        0   

     half_trip_id_54137203.0  half_trip_id_54137204.0  \
729                        0                        0   
730                        0                        0   
731                        0                   

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Define a custom dataset class for PyTorch:
class CustomDataset(Dataset):
    def __init__(self, df):
        self.df = df
        self.n_features = len(df.columns) - 1  # number of input features

    def __len__(self):
        return len(self.df)

    def __getitem__(self, index):
        X = self.df.iloc[index, :-1].values  # input sequence
        y = self.df.iloc[index, -1]          # output target
        X = np.expand_dims(X, axis=0)        # add a batch dimension
        return torch.FloatTensor(X), torch.FloatTensor([y.item()])

# Define a collate function for padding sequences:
def custom_collate(batch):
    X_batch = [item[0] for item in batch]
    y_batch = [item[1] for item in batch]
    X_padded = torch.nn.utils.rnn.pad_sequence(X_batch, batch_first=True, padding_value=0)
    y_padded = torch.nn.utils.rnn.pad_sequence(y_batch, batch_first=True, padding_value=0)
    return X_padded, y_padded

# Create a DataLoader using the custom dataset and collate function:
dataset = CustomDataset(one_hot_encoded_data)
dataloader = DataLoader(dataset, batch_size=30, collate_fn=custom_collate, shuffle=True)

In [ ]:
# Define the training function:
def train(model, dataloader, criterion, optimizer):
    #model.train()
    running_loss = 0.0
    for i, (inputs, labels) in enumerate(dataloader):
        optimizer.zero_grad()
        print("here")
        print(inputs.size())
        inputs = inputs[:, -1, :]
        #inputs = inputs.view(129,30)
        print(inputs.size())
        outputs = model(inputs[i].unsqueeze(0))
        print("here")
        loss = criterion(outputs, labels)
        print("here")
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    return running_loss / len(dataloader)

In [ ]:
# Define the evaluation function:
def evaluate(model, dataloader, criterion):
    model.eval()
    running_loss = 0.0
    with torch.no_grad():
        for i, batch in enumerate(dataloader):
            inputs, labels = batch
            inputs_padded = torch.nn.utils.rnn.pad_sequence(inputs, batch_first=True, padding_value=0)
            outputs = model(inputs_padded)
            loss = criterion(outputs, labels)
            running_loss += loss.item()
    return running_loss / len(dataloader)

In [ ]:
# Scale the dataset using the StandardScaler and split it into train and test sets:
df2 = one_hot_encoded_data
scaler = StandardScaler()
df2[df2.columns] = scaler.fit_transform(df2[df2.columns])
X = df2.values
y = Y.values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Convert the data into PyTorch tensors and create the dataloaders
X_train_tensor = torch.tensor(X_train).float()
y_train_tensor = torch.tensor(y_train).float()
train_dataset = CustomDataset(pd.DataFrame(np.concatenate((X_train, y_train.reshape(-1, 1)), axis=1)))
train_dataloader = DataLoader(train_dataset, batch_size=30, collate_fn=custom_collate, shuffle=True)
X_test_tensor = torch.tensor(X_test).float()
y_test_tensor = torch.tensor(y_test).float()
test_dataset = CustomDataset(pd.DataFrame(np.concatenate((X_test, y_test.reshape(-1, 1)), axis=1)))
test_dataloader = DataLoader(test_dataset, batch_size=30, collate_fn=custom_collate, shuffle=False)

In [ ]:
criterion = nn.MSELoss()
optimizer = optim.Adam(ntm.parameters(), lr=0.001)

In [ ]:
# Train the model
num_epochs = 50
train_losses = []
test_losses = []
for epoch in range(num_epochs):
    ntm.reset()
    train_loss = train(ntm, train_dataloader, criterion, optimizer)
    test_loss = evaluate(ntm, test_dataloader, criterion)
    train_losses.append(train_loss)
    test_losses.append(test_loss)
    print(f'Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}, Test Loss: {test_loss:.4f}')

here
torch.Size([30, 1, 129])
torch.Size([30, 129])


RuntimeError: ignored

In [ ]:
from google.colab import drive
drive.mount('drive')
df2.to_csv('df2Data.csv')
!cp df2Data.csv "drive/My Drive/"

Drive already mounted at drive; to attempt to forcibly remount, call drive.mount("drive", force_remount=True).


In [ ]:
from tqdm import tqdm


# ==== Training Settings ====
# Loss Function
criterion = nn.MSELoss()
optimizer = optim.SGD(ntm.parameters(),lr=1e-4)

# optimizer = optim.Adam(ntm.parameters(), lr=args.lr,
#                        betas=(args.beta1, args.beta2))

# ==== Training ====
losses = []
errors = []

for step in tqdm(range(args.num_steps)):
    
    optimizer.zero_grad()
    ntm.reset()
    
    # Sample data
    data = dataset[step]
    inputs, target = data['input'], data['target']
    
    # Tensor to store outputs
    out = torch.zeros(target.size())
    
    # Process the inputs through NTM for memorization
    for i in range(inputs.size()[0]):
        # Forward passing all sequences for read
        ntm(inputs[i].unsqueeze(0))
        
    # Get the outputs from memory without real inputs
    zero_inputs = torch.zeros(inputs.size()[1]).unsqueeze(0) # dummy inputs
    for i in range(target.size()[0]):
        out[i] = ntm(zero_inputs)
    
    # Compute loss, backprop, and optimize
    loss = criterion(out, target)
    losses.append(loss.item())
    loss.backward()
    nn.utils.clip_grad_value_(ntm.parameters(), 10)
    optimizer.step()
    
    # Calculate binary outputs
    binary_output = out.clone()
    binary_output = binary_output.detach().apply_(lambda x: 0 if x < 0.5 else 1)
    
    # Sequence prediction error is calculted in bits per sequence
    error = torch.sum(torch.abs(binary_output - target))
    errors.append(error.item())
    
    # Print Stats
    if step % 200 == 0:
        print('Step {} == Loss {:.3f} == Error {} bits per sequence'.format(step, np.mean(losses), np.mean(errors)))
        losses = []
        errors = []
        
# Save model
torch.save(ntm.state_dict(), args.saved_model)

In [ ]:
df3=pd.read_csv("MBTA-Bus-Arrival-Departure-Times_2022-01.csv", parse_dates=["service_date"])
df=df3[0:4000]
day_of_week = df['service_date'].dt.day_name()
df.loc[:, 'day_of_week'] = day_of_week
df['weekday'] = df['service_date'].dt.day_of_week
df['day_type'] = df['service_date'].apply(lambda x: 'Weekend' if x.weekday() >= 5 else 'Weekday')
df['day_type']=df.day_type.factorize()[0]+ 1
df['scheduled_time']=pd.to_datetime(df['scheduled'], errors='coerce')
tindex=pd.DatetimeIndex(df.scheduled_time)
df['scheduled_time']=tindex.strftime('%H:%M:%S')
df['converted_scheduled_time']=pd.to_timedelta(df['scheduled_time']).dt.total_seconds()
df['actual_time']=pd.to_datetime(df['actual'], errors='coerce')
tindex=pd.DatetimeIndex(df.actual_time)
df['actual_time']=tindex.strftime('%H:%M:%S')
df['converted_actual_time']=pd.to_timedelta(df['actual_time']).dt.total_seconds()
features=['service_date','route_id', 'direction_id', 'half_trip_id', 'stop_id', 'time_point_id', 'time_point_order', 'point_type', 'day_type', 'converted_scheduled_time', 'converted_actual_time']
df=df[features]
new_df = df.dropna()
new_df['time_difference']=new_df['converted_actual_time'] - new_df['converted_scheduled_time']
new_df['late_early_status'] = np.where(new_df['time_difference']> 0, True, False)
conditions = [
    (new_df['converted_actual_time'] > 25200) & (new_df['converted_actual_time'] <= 32400),
    (new_df['converted_actual_time'] > 57600) & (new_df['converted_actual_time'] <= 64800)
    ]
values = [1,1]
new_df['rush_hour'] = np.select(conditions, values)
features=['service_date','route_id','half_trip_id','time_point_order', 'point_type', 'day_type', 'time_difference','rush_hour']
new_df=new_df[features]


<ipython-input-81-806989cbf447>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:, 'day_of_week'] = day_of_week
<ipython-input-81-806989cbf447>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['weekday'] = df['service_date'].dt.day_of_week
<ipython-input-81-806989cbf447>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/s

In [ ]:
new_df.head(10)

,service_date,route_id,half_trip_id,time_point_order,point_type,day_type,time_difference,rush_hour
0,2022-01-01,01,54137355.0,1,Startpoint,1,1196.0,0
1,2022-01-01,01,54137355.0,2,Midpoint,1,1126.0,0
2,2022-01-01,01,54137355.0,3,Midpoint,1,1216.0,0
3,2022-01-01,01,54137355.0,4,Midpoint,1,1203.0,0
4,2022-01-01,01,54137355.0,5,Midpoint,1,1150.0,0
5,2022-01-01,01,54137355.0,6,Midpoint,1,1300.0,0
6,2022-01-01,01,54137355.0,7,Midpoint,1,1239.0,0
7,2022-01-01,01,54137203.0,1,Startpoint,1,53.0,0
8,2022-01-01,01,54137203.0,2,Midpoint,1,-68.0,0
9,2022-01-01,01,54137355.0,8,Midpoint,1,1247.0,0


In [ ]:
# Scale the dataset using the StandardScaler and split it into train and test sets:
scaler = StandardScaler()
df = new_df[new_df.columns[1:8]]
print(df.head(10))
df = pd.get_dummies(df, columns = ['route_id','half_trip_id','time_point_order','point_type'])
df[df.columns] = scaler.fit_transform(df[df.columns])
X = df.drop('time_difference', axis=1).values
y = df['time_difference'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert the data into PyTorch tensors and create the dataloaders
X_train_tensor = torch.tensor(X_train).float()
y_train_tensor = torch.tensor(y_train).float()
train_dataset = CustomDataset(pd.DataFrame(np.concatenate((X_train, y_train.reshape(-1, 1)), axis=1)))
train_dataloader = DataLoader(train_dataset, batch_size=14, collate_fn=custom_collate, shuffle=True)
X_test_tensor = torch.tensor(X_test).float()
y_test_tensor = torch.tensor(y_test).float()
test_dataset = CustomDataset(pd.DataFrame(np.concatenate((X_test, y_test.reshape(-1, 1)), axis=1)))
test_dataloader = DataLoader(test_dataset, batch_size=14, collate_fn=custom_collate, shuffle=False)

  route_id  half_trip_id  time_point_order  point_type  day_type  \
0       01    54137355.0                 1  Startpoint         1   
1       01    54137355.0                 2    Midpoint         1   
2       01    54137355.0                 3    Midpoint         1   
3       01    54137355.0                 4    Midpoint         1   
4       01    54137355.0                 5    Midpoint         1   
5       01    54137355.0                 6    Midpoint         1   
6       01    54137355.0                 7    Midpoint         1   
7       01    54137203.0                 1  Startpoint         1   
8       01    54137203.0                 2    Midpoint         1   
9       01    54137355.0                 8    Midpoint         1   

   time_difference  rush_hour  
0           1196.0          0  
1           1126.0          0  
2           1216.0          0  
3           1203.0          0  
4           1150.0          0  
5           1300.0          0  
6           1239.0         

In [ ]:
criterion = nn.MSELoss()
optimizer = optim.Adam(ntm.parameters(), lr=0.001)

# Train the model
num_epochs = 50
train_losses = []
test_losses = []
for epoch in range(num_epochs):
    ntm.reset()
    train_loss = train(ntm, train_dataloader, criterion, optimizer)
    test_loss = evaluate(ntm, test_dataloader, criterion)
    train_losses.append(train_loss)
    test_losses.append(test_loss)
    print(f'Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}, Test Loss: {test_loss:.4f}')

here
torch.Size([14, 1, 552])
torch.Size([14, 552])


RuntimeError: ignored

In [ ]:
from google.colab import drive
drive.mount('drive')
df.to_csv('NargesData.csv')
!cp NargesData.csv "drive/My Drive/"

Drive already mounted at drive; to attempt to forcibly remount, call drive.mount("drive", force_remount=True).
